# classification

## Create dataset from file

In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.datasets import dump_svmlight_file
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics

def build_model(name, class_num):
    
    path = '/home/kevinxing/Documents/BigData/Dota2_Player_Classification/'
    
    
    file_name = path + name + ".csv"
    svm_file = name + "_svm.txt"
    model_name = name + ".model"
    
    X = []
    Y = []
    f = open(file_name, "r")
    for line in f:
        line = line.strip('\r\n')
        line = line.split(',')
        x_tmp = []
        for i in range(len(line)):
            if i != len(line) - 1 :
                x_tmp.append(float(line[i]))
            else:
                #print x_tmp
                X.append(x_tmp)
                Y.append(int(float(line[i])))
                
    fsvm = open(svm_file, 'w')
    dump_svmlight_file(X, Y, fsvm, zero_based=False)
    fsvm.close()
    
    # $example off$
    #from pyspark import SparkContext
    #sc = SparkContext(appName="MultiClassMetricsExample")

    # Several of the methods available in scala are currently missing from pyspark
    # $example on$
    # Load training data in LIBSVM format
    data = MLUtils.loadLibSVMFile(sc, svm_file)

    # Split data into training (60%) and test (40%)
    training, test = data.randomSplit([0.6, 0.4], seed=11)
    training.cache()

    # Run training algorithm to build the model
    model = LogisticRegressionWithLBFGS.train(training, numClasses=class_num)

    # Compute raw scores on the test set
    predictionAndLabels = test.map(lambda lp: (float(model.predict(lp.features)), lp.label))

    # Instantiate metrics object
    metrics = MulticlassMetrics(predictionAndLabels)

    # Overall statistics
    precision = metrics.precision()
    recall = metrics.recall()
    f1Score = metrics.fMeasure()
    print(name + "Summary Stats")
    print("Precision = %s" % precision)
    print("Recall = %s" % recall)
    print("F1 Score = %s" % f1Score)

    # Statistics by class
    labels = data.map(lambda lp: lp.label).distinct().collect()
    for label in sorted(labels):
        print("Class %s precision = %s" % (label, metrics.precision(label)))
        print("Class %s recall = %s" % (label, metrics.recall(label)))
        print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))

    # Weighted stats
    print("Weighted recall = %s" % metrics.weightedRecall)
    print("Weighted precision = %s" % metrics.weightedPrecision)
    print("Weighted F(1) Score = %s" % metrics.weightedFMeasure())
    print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
    print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)
    
    model.save(sc, model_name)

#print a,
#print raw_data
#sc = SparkContext('local')
#spark = SparkSession(sc)
#labels = ['pregnant_times', 'plasma_glucose', 'blood_pressure', 'skin_fold', 'serum_insulin', 'body_mass', 'diabates', 'age']
#data = spark.createDataFrame(raw_data, labels)
#data.show()
#print X
#print Y

## Create svm file

In [14]:
#build_model("1_players_model", 4)
#build_model("2_players_model", 5)
#build_model("3_players_model", 4)
#build_model("4_players_model", 5)
#build_model("5_players_model", 5)
#build_model("6_players_model", 4)
#build_model("7_players_model", 4)
#build_model("8_players_model", 5)
#build_model("9_players_model", 5)
#build_model("10_players_model", 4)



#print Y
#fsvm = open('players_info_kda.txt', 'w')
#dump_svmlight_file(X, Y, fsvm, zero_based=False)
#fsvm.close()

10_players_modelSummary Stats
Precision = 0.293603703462
Recall = 0.293603703462
F1 Score = 0.293603703462
Class 0.0 precision = 0.303317535545
Class 0.0 recall = 0.0139646519747
Class 0.0 F1 Measure = 0.0267000417188
Class 1.0 precision = 0.4537299339
Class 1.0 recall = 0.211720643314
Class 1.0 F1 Measure = 0.288718642031
Class 2.0 precision = 0.305234110735
Class 2.0 recall = 0.795407098121
Class 2.0 F1 Measure = 0.441170794468
Class 3.0 precision = 0.0276211950395
Class 3.0 recall = 0.0257352941176
Class 3.0 F1 Measure = 0.0266449157151
Weighted recall = 0.293603703462
Weighted precision = 0.314144842023
Weighted F(1) Score = 0.220739350711
Weighted F(0.5) Score = 0.228152546011
Weighted false positive rate = 0.250709906909


In [73]:
#
# Licensed to the Apache Software Foundation (ASF) under one or more
# contributor license agreements.  See the NOTICE file distributed with
# this work for additional information regarding copyright ownership.
# The ASF licenses this file to You under the Apache License, Version 2.0
# (the "License"); you may not use this file except in compliance with
# the License.  You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

# $example on$
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics
# $example off$

from pyspark import SparkContext


#sc = SparkContext(appName="MultiClassMetricsExample")

    # Several of the methods available in scala are currently missing from pyspark
    # $example on$
    # Load training data in LIBSVM format
data = MLUtils.loadLibSVMFile(sc, "players_info_kda.txt")

    # Split data into training (60%) and test (40%)
training, test = data.randomSplit([0.7, 0.3], seed=11)
training.cache()

    # Run training algorithm to build the model
model = LogisticRegressionWithLBFGS.train(training, numClasses=9)

    # Compute raw scores on the test set
predictionAndLabels = test.map(lambda lp: (float(model.predict(lp.features)), lp.label))

    # Instantiate metrics object
metrics = MulticlassMetrics(predictionAndLabels)

    # Overall statistics
precision = metrics.precision()
recall = metrics.recall()
f1Score = metrics.fMeasure()
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)

    # Statistics by class
labels = data.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))
    print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))

    # Weighted stats
print("Weighted recall = %s" % metrics.weightedRecall)
print("Weighted precision = %s" % metrics.weightedPrecision)
print("Weighted F(1) Score = %s" % metrics.weightedFMeasure())
print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)
    # $example off$


/usr/local/lib/python2.7/dist-packages/pyspark/mllib/evaluation.py:237: UserWarning: Deprecated in 2.0.0. Use accuracy.
  warnings.warn("Deprecated in 2.0.0. Use accuracy.")
/usr/local/lib/python2.7/dist-packages/pyspark/mllib/evaluation.py:249: UserWarning: Deprecated in 2.0.0. Use accuracy.
  warnings.warn("Deprecated in 2.0.0. Use accuracy.")
/usr/local/lib/python2.7/dist-packages/pyspark/mllib/evaluation.py:262: UserWarning: Deprecated in 2.0.0. Use accuracy.
  warnings.warn("Deprecated in 2.0.0. Use accuracy.")


Summary Stats
Precision = 0.557831561093
Recall = 0.557831561093
F1 Score = 0.557831561093
Class 0.0 precision = 0.559681697613
Class 0.0 recall = 0.691803278689
Class 0.0 F1 Measure = 0.618768328446
Class 1.0 precision = 0.428186714542
Class 1.0 recall = 0.20902716915
Class 1.0 F1 Measure = 0.280918727915
Class 2.0 precision = 0.512494705633
Class 2.0 recall = 0.643617021277
Class 2.0 F1 Measure = 0.57062013676
Class 3.0 precision = 0.575163398693
Class 3.0 recall = 0.59620596206
Class 3.0 F1 Measure = 0.585495675316
Class 4.0 precision = 0.586552217454
Class 4.0 recall = 0.793036750484
Class 4.0 F1 Measure = 0.674342105263
Class 5.0 precision = 0.675287356322
Class 5.0 recall = 0.714285714286
Class 5.0 F1 Measure = 0.69423929099
Class 6.0 precision = 0.384615384615
Class 6.0 recall = 0.320600272851
Class 6.0 F1 Measure = 0.349702380952
Class 7.0 precision = 0.477021524142
Class 7.0 recall = 0.339123242349
Class 7.0 F1 Measure = 0.396422528402
Class 8.0 precision = 0.626674786845
Clas

In [76]:
print type(model)
model.predict([0,0,1])

<class 'pyspark.mllib.classification.LogisticRegressionModel'>


6

In [77]:
model.save(sc, "test_model.model")

In [79]:
model2 = LogisticRegressionModel.load(sc, "test_model.model")


In [80]:
model2.predict([0,0,1])

6